In [2]:
import dataiku
import os
import requests
import asyncio
import nest_asyncio
from datetime import datetime, date, timezone
from typing import List, Dict, Optional
import json
from pydantic import BaseModel, Field
from fastmcp import Client
from mcp.server.fastmcp import FastMCP, Context
from zoneinfo import ZoneInfo
nest_asyncio.apply()

# schema models --------------------------
class DailyForecast(BaseModel):
    date: date
    temp_min: float
    temp_max: float
    condition: str

class CurrentConditions(BaseModel):
    temp: float
    wind_speed: float
    condition: str
    alerts: List[str] = Field(default_factory=list)

class HourlyForecast(BaseModel):
    dt: int
    temp: float
    condition: str

class CityWeather(BaseModel):
    city: str
    state: str
    current_conditions: Optional[CurrentConditions] = None
    hourly_forecast: Optional[List[HourlyForecast]] = None
    forecast: Optional[List[DailyForecast]] = None
    overview: Optional[str] = None
    timestamp_data: Optional[dict] = None
    aggregation_data: Optional[dict] = None

# Utility Function --------------------------
def _get_coords(city: str, state: str, api_key: str):
    r = requests.get(
        "http://api.openweathermap.org/geo/1.0/direct",
        params={"q": f"{city},{state},US", "limit": 1, "appid": api_key}
    )
    r.raise_for_status()
    data = r.json()
    if not data:
        raise ValueError(f"No location found for {city}, {state}")
    return {"lat": data[0]["lat"], "lon": data[0]["lon"]}

# MCP Server & Tool --------------------------

def get_api_key(secret_name: str = "API_KEY_EST1-CFN-AI-EDA-DEV-01") -> str:
    """Retrieves an API key from Dataiku user secrets."""
    for secret in dataiku.api_client().get_auth_info(with_secrets=True)["secrets"]:
        if secret["key"] == secret_name:
            return secret["value"]
    raise KeyError(f"User secret key {secret_name} not found.")

OPENWEATHER_API_KEY = get_api_key("OPENWEATHER_API_KEY")

mcp = FastMCP("WeatherToolSingle")

@mcp.tool()
async def get_weather_for_cities(
    locations: List[Dict[str, str]],
    endpoint_type: str = "current",
    hourly_count: Optional[int] = 8,
    timestamp: Optional[int] = None,
    date_str: Optional[str] = None,
    ctx: Context = None
) -> List[CityWeather]:
    """
    Fetch weather data for a list of U.S. cities using OpenWeather One Call API 3.0.

    endpoint_type: one of:
      • "current": returns current conditions, minute forecast (1 h),
        hourly forecast (up to 48 h), daily forecast (up to 8 days), and weather alerts.
      • "timestamp": returns weather for a specific UNIX timestamp
        (historical or future forecast, up to 4 days ahead).
      • "aggregation": returns aggregated daily data for a given date
        (history from 1979 or forecast up to ~1.5 years).
      • "overview": returns an AI‑generated human‑readable summary for today and tomorrow.

    Parameters:
      - locations (list): [{"city": str, "state": str}], required.
      - endpoint_type (str): required. See valid types above.
      - hourly_count (int): used only for "current"; number of hours (default 8, max 48).
      - timestamp (int): UNIX seconds, required for "timestamp" endpoint.
      - date_str (str): YYYY‑MM‑DD string, required for "aggregation" endpoint.

    Returns: list of `CityWeather` objects with fields depending on endpoint_type:
      - "current": includes `.current_conditions`, `.hourly_forecast`, `.forecast`, `.alerts`.
      - "timestamp": includes `.timestamp_data`.
      - "aggregation": includes `.aggregation_data`.
      - "overview": includes `.overview` (string summary).

    Examples:

      Current:
        {
          "locations": [{"city": "Greenville", "state": "SC"}],
          "endpoint_type": "current",
          "hourly_count": 8
        }

      Timestamp:
        {
          "locations": [{"city": "Greenville", "state": "SC"}],
          "endpoint_type": "timestamp",
          "timestamp": 1712145600
        }

      Aggregation:
        {
          "locations": [{"city": "Greenville", "state": "SC"}],
          "endpoint_type": "aggregation",
          "date_str": "2025-07-01"
        }

      Overview:
        {
          "locations": [{"city": "Greenville", "state": "SC"}],
          "endpoint_type": "overview"
        }
    """


    await ctx.info(f"Invoked endpoint={endpoint_type}, hourly_count={hourly_count}")
    results = []
    for loc in locations:
        city, state = loc["city"], loc["state"]
        try:
            coords = _get_coords(city, state, OPENWEATHER_API_KEY)
            base = "https://api.openweathermap.org/data/3.0"
            params = {
                "lat": coords["lat"], "lon": coords["lon"],
                "appid": OPENWEATHER_API_KEY, "units": "imperial"
            }

            if endpoint_type == "current":
                resp = requests.get(f"{base}/onecall", params=params)
                resp.raise_for_status()
                jd = resp.json()
                current = CurrentConditions(
                    temp=jd["current"]["temp"],
                    wind_speed=jd["current"]["wind_speed"],
                    condition=jd["current"]["weather"][0]["description"],
                    alerts=[a["event"] for a in jd.get("alerts", [])]
                )
                hourly = jd.get("hourly", [])[: (hourly_count or 8)]
                hourly_models = [
                    HourlyForecast(dt=h["dt"], temp=h["temp"],
                                   condition=h["weather"][0]["description"])
                    for h in hourly
                ]
                daily = [
                    DailyForecast(
                        date=datetime.fromtimestamp(d["dt"]).date(),
                        temp_min=d["temp"]["min"],
                        temp_max=d["temp"]["max"],
                        condition=d["weather"][0]["description"]
                    ) for d in jd.get("daily", [])[:8]
                ]
                cw = CityWeather(
                    city=city, state=state,
                    current_conditions=current,
                    hourly_forecast=hourly_models,
                    forecast=daily
                )

            elif endpoint_type == "timestamp":
                if timestamp is None:
                    raise ValueError("timestamp required for timestamp endpoint")
                resp = requests.get(f"{base}/onecall/timemachine",
                                    params={**params, "dt": timestamp})
                resp.raise_for_status()
                cw = CityWeather(city=city, state=state,
                                 timestamp_data=resp.json())

            elif endpoint_type == "aggregation":
                if not date_str:
                    raise ValueError("date_str required for aggregation endpoint")
                resp = requests.get(f"{base}/onecall/day_summary",
                                    params={**params, "date": date_str})
                resp.raise_for_status()
                cw = CityWeather(city=city, state=state,
                                 aggregation_data=resp.json())

            elif endpoint_type == "overview":
                resp = requests.get(f"{base}/onecall/overview", params=params)
                resp.raise_for_status()
                cw = CityWeather(city=city, state=state,
                                 overview=resp.json().get("summary"))

            else:
                raise ValueError(f"Unsupported endpoint_type '{endpoint_type}'")

            results.append(cw)

        except Exception as e:
            results.append(CityWeather(city=city, state=state,
                                       overview=f"Error: {e}"))

    return results

# in memory client test --------------------------
client = Client(mcp)

async def test_call():
    async with client:
        print("Tools:", await client.list_tools())
        return await client.call_tool("get_weather_for_cities", {
          "locations": [{"city": "Greenville", "state": "SC"}],
          "endpoint_type": "current",
          "hourly_count": 8
        }
)

final = asyncio.get_event_loop().run_until_complete(test_call())

/opt/dataiku/installer/dataiku-dss-12.6.2/python/dataiku/base/utils.py:206: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if min_version is not None and LooseVersion(p.__version__) < LooseVersion(min_version):
/opt/dataiku/installer/dataiku-dss-12.6.2/python/dataiku/core/platform_exec.py:7: DeprecationWarning: 'pipes' is deprecated and slated for removal in Python 3.13
  import pipes


Tools: [Tool(name='get_weather_for_cities', description='\n    Fetch weather data for a list of U.S. cities using OpenWeather One Call API 3.0.\n\n    endpoint_type: one of:\n      • "current": returns current conditions, minute forecast (1\u202fh),\n        hourly forecast (up to 48\u202fh), daily forecast (up to 8 days), and weather alerts.\n      • "timestamp": returns weather for a specific UNIX timestamp\n        (historical or future forecast, up to 4 days ahead).\n      • "aggregation": returns aggregated daily data for a given date\n        (history from 1979 or forecast up to ~1.5 years).\n      • "overview": returns an AI‑generated human‑readable summary for today and tomorrow.\n\n    Parameters:\n      - locations (list): [{"city": str, "state": str}], required.\n      - endpoint_type (str): required. See valid types above.\n      - hourly_count (int): used only for "current"; number of hours (default\u202f8, max\u202f48).\n      - timestamp (int): UNIX seconds, required for

In [7]:
content_obj = final[0] 
raw_json_str = content_obj.text
data = json.loads(raw_json_str)

def to_est(ts):
    utc_dt = datetime.fromtimestamp(ts, tz=timezone.utc)
    est_dt = utc_dt.astimezone(ZoneInfo("America/New_York"))
    return est_dt.strftime("%Y-%m-%d %I:%M %p %Z")

def display_current(data):
    cc = data.get("current_conditions", {}) or {}
    print(f"📍 {data['city']}, {data['state']}")
    print(f"Now: {cc.get('temp')}°F — {cc.get('condition')}, Wind: {cc.get('wind_speed')} mph")
    alerts = cc.get("alerts", [])
    print("Alerts:", "; ".join(alerts) if alerts else "None")
    print("\n🕒 Hourly Forecast (Eastern Time):")
    for h in data.get("hourly_forecast", []):
        print(f"  • {to_est(h['dt'])}: {h['temp']}°F — {h['condition']}")
    print("\n8‑Day Forecast:")
    for d in data.get("forecast", []):
        print(f"  • {d['date']}: High {d['temp_max']}°F / Low {d['temp_min']}°F — {d['condition']}")

def display_aggregation(data):
    agg = data.get("aggregation_data", {}) or {}
    print(f"📍 {data['city']}, {data['state']} — Aggregated for {agg.get('date')}")
    tz = agg.get("tz")
    if tz:
        print(f"Timezone offset: {tz}")
    for key, section in agg.items():
        if isinstance(section, dict):
            print(f"\n**{key.capitalize()}**")
            for sub, val in section.items():
                if sub in ("start","end") and isinstance(val, int):
                    print(f"  {sub}: {to_est(val)}")
                else:
                    print(f"  {sub}: {val}")

def display_overview(data):
    print(f"{data['city']}, {data['state']}")
    print("Overview:", data.get("overview", "(none)"))

def display_timestamp(data):
    tsd = data.get("timestamp_data") or {}
    print(f"{data['city']}, {data['state']}")
    print("Timestamp Data (raw JSON):")
    print(json.dumps(tsd, indent=2))

def display(weather_json_str, endpoint_type):
    data = json.loads(weather_json_str)
    if endpoint_type == "current":
        display_current(data)
    elif endpoint_type == "aggregation":
        display_aggregation(data)
    elif endpoint_type == "overview":
        display_overview(data)
    elif endpoint_type == "timestamp":
        display_timestamp(data)
    else:
        print("Unknown endpoint_type")

content = final[0]
display(content.text, "current")

📍 Greenville, SC
Now: 94.32°F — clear sky, Wind: 10.36 mph
Alerts: None

🕒 Hourly Forecast (Eastern Time):
  • 2025-07-08 01:00 PM EDT: 93.85°F — clear sky
  • 2025-07-08 02:00 PM EDT: 94.32°F — clear sky
  • 2025-07-08 03:00 PM EDT: 94.12°F — clear sky
  • 2025-07-08 04:00 PM EDT: 94.08°F — clear sky
  • 2025-07-08 05:00 PM EDT: 93.97°F — clear sky
  • 2025-07-08 06:00 PM EDT: 92.75°F — clear sky
  • 2025-07-08 07:00 PM EDT: 90.93°F — few clouds
  • 2025-07-08 08:00 PM EDT: 87.73°F — scattered clouds

8‑Day Forecast:
  • 2025-07-08: High 94.32°F / Low 71.01°F — clear sky
  • 2025-07-09: High 95.52°F / Low 70.68°F — light rain
  • 2025-07-10: High 94.93°F / Low 68.59°F — moderate rain
  • 2025-07-11: High 95.16°F / Low 68.31°F — moderate rain
  • 2025-07-12: High 95.77°F / Low 70.7°F — light rain
  • 2025-07-13: High 99.12°F / Low 70.0°F — light rain
  • 2025-07-14: High 99.1°F / Low 72.3°F — light rain
  • 2025-07-15: High 97.81°F / Low 72.05°F — light rain
